In [1]:
#Install inbuilt Wikipedia Tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [2]:
#Wrapper around Wikipedia API
wiki_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool1=WikipediaQueryRun(api_wrapper=wiki_wrapper)
tool1

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data Science\\Projects\\Gen AI\\Project Lang Chain Q&A\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [3]:
tool1.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [4]:
tool1.run("Max Verstappen")

'Page: Max Verstappen\nSummary: Max Emilian Verstappen (Dutch pronunciation: [ˈmɑks vɛrˈstɑ.pə(n)]; born 30 September 1997) is a Belgian and Dutch racing driver competing in Formula One under the Dutch '

Custom Tool using Web Url

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [6]:
#Loading from web
loader=WebBaseLoader("https://www.formula1.com/")
docs=loader.load()
docs

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\nF1 - The Official Home of Formula 1® Racing\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to Global Nav\nSkip to Primary content\nSkip to main content\n\n\n\n\n\n  Formula1 ® All F1 news and results  Install\n\n\n\nToggle Navigation>\n\n\n\n\n\n\n\nLogin/Register\nUser Options\n\n\n\n\n\n\n\n\n\n\n\n\n\nLatest\n\n\n\n\n\n\nLatest\nWhat is F1? \nWhat is F1 TV? \nGet involved \n\n\n\n\n\n\n\n\n\n\nVideo\n\n\n\nF1 Unlocked\n\n\n\nSchedule\n\n\n\n\n\n\nFull Schedule \n2025 \nRace Programmes \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nResults\n\n\n\n\n\n\n2024 Season \nDriver Standings \nConstructor Standings \nArchive 1950-2023 \nF1 Awards \n\n\n\n\n\n\n\nDrivers\n\n\n\n\n\n\nAll Drivers \nHall of Fame \n\n\n\n\n\n\nAlexander\nAlbon\n\n\n\n\n\n\n\nFernando\nAlonso\n\n\n\n\n\n\n\nValtteri\nBottas\n\n\n\n\n\n\n\nPierre\n

In [8]:
#Transforming into chunks
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

#Converting into vectors and storing in vector store
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())

Creating a retriever tool

In [9]:
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021F493E1FF0>)

In [10]:
from langchain.tools.retriever import create_retriever_tool
prompt="Search for information about Formula1 information. It can be about Drivers,Cars or Championships"
tool2=create_retriever_tool(retriever,"Formula1 Search",prompt)

In [11]:
tool2.name

'Formula1 Search'

In [13]:
#Combining both the tools
tools=[tool1,tool2]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Data Science\\Projects\\Gen AI\\Project Lang Chain Q&A\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Formula1 Search', description='Search for information about Formula1 information. It can be about Drivers,Cars or Championships', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021F3ABFE830>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021F493E1FF0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000021F3ABFE9E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'

Constructing an Agent

In [15]:
#Prompts from Hub 
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [17]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [18]:
#LLM Model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [19]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

In [20]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c1867281

In [ ]:
result = agent_executor.invoke({"input": "Tell me about Max Verstappen"})